![aga](img/AB_logo.png)

# Introduction to Schema Objects
***
A database **schema** is a logical container for data structures, called schema objects. Examples of schema objects are tables and indexes. You create and manipulate schema objects with SQL.

A **[database user](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-2D2B8165-D2CE-4D08-82D6-A7E1D988471B)** account has a password and specific database privileges. Each user account owns a single schema, which has the same name as the user. The schema contains the data for the user owning the schema. For example, the `hr` user account owns the `hr` schema, which contains schema objects such as the `employees` table. In a production database, the schema owner usually represents a database application rather than a person.

Within a schema, each schema object of a particular type has a unique name. For example, `hr.employees` refers to the table `employees` in the `hr` schema. Figure 2-1 depicts a schema owner named `hr` and schema objects within the `hr` schema.

***Figure 2-1 HR Schema***
![](https://docs.oracle.com/database/121/CNCPT/img/GUID-35D3B6DB-6F3B-42A0-BA5B-9CA8BD2FABF2-default.gif)

## Schema Object Types

Oracle SQL enables you to create and manipulate many other types of schema objects.

The principal types of schema objects are:

* Tables

  A **[table](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-DA8F7E11-B6AF-4ED3-B2A9-B5741E9AE2D4)** stores data in rows. Tables are the most important schema objects in a relational database.

* Indexes

  Indexes are schema objects that contain an entry for each indexed row of the table or **[table cluster](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-1C56177E-6BEE-4FE7-B45E-38298CDB946D)** and provide direct, fast access to rows. Oracle Database supports several types of index. An **[index-organized table](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-FD628CF1-3A67-4171-92BB-D4A74119ACD9)** is a table in which the data is stored in an index structure.

* Partitions

  Partitions are pieces of large tables and indexes. Each partition has its own name and may optionally have its own storage characteristics.

* Views

  Views are customized presentations of data in one or more tables or other views. You can think of them as stored queries. Views do not actually contain data.

* Sequences

  A sequence is a user-created object that can be shared by multiple users to generate integers. Typically, you use sequences to generate **[primary key](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-8640EFA5-276C-4812-A078-1F21F55F4200)** values.

* Dimensions

  A dimension defines a parent-child relationship between pairs of column sets, where all the columns of a column set must come from the same table. Dimensions are commonly used to categorize data such as customers, products, and time.

* Synonyms

  A synonym is an alias for another schema object. Because a synonym is simply an alias, it requires no storage other than its definition in the **[data dictionary](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-D6A3934F-BA6D-464F-9612-C683E20514A4)**.

* PL/SQL subprograms and packages

  PL/SQL is the Oracle procedural extension of SQL. A **[PL/SQL subprogram](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-4BF65C07-E5DD-454C-8AD0-E38EF5672CE4)** is a named PL/SQL block that can be invoked with a set of parameters. A PL/SQL package groups logically related PL/SQL types, variables, and subprograms.

Other types of objects are also stored in the database and can be created and manipulated with SQL statements but are not contained in a schema. These objects include database user account, roles, contexts, and dictionary objects.

## Schema Object Storage

Some schema objects store data in a type of logical storage structure called a **segment**. For example, a nonpartitioned heap-organized table or an index creates a segment.

Other schema objects, such as views and sequences, consist of metadata only. This topic describes only schema objects that have segments.

Oracle Database stores a schema object logically within a **[tablespace](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-AA66891C-71B2-4D55-8F64-0E427AE24E88)**. There is no relationship between schemas and tablespaces: a tablespace can contain objects from different schemas, and the objects for a schema can be contained in different tablespaces. The data of each object is physically contained in one or more data files.

The following figure shows a possible configuration of table and index segments, tablespaces, and data files. The data segment for one table spans two data files, which are both part of the same tablespace. A segment cannot span multiple tablespaces.

***Figure 2-2 Segments, Tablespaces, and Data Files***
![Figure 2-2 Segments, Tablespaces, and Data Files](https://docs.oracle.com/database/121/CNCPT/img/GUID-C650BC57-BCBB-46EC-A92F-7CA2319C80D2-default.gif)

## Schema Object Dependencies

Some schema objects refer to other objects, creating a **schema object dependency**.

For example, a view contains a **[query](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-CCF91C9F-A98A-498F-A84B-58A0FA16CD6E)** that references tables or views, while a **[PL/SQL](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-68532EDC-3324-48DE-9AD5-52CD2EC671EA)** subprogram invokes other subprograms. If the definition of object A references object B, then A is a **[dependent object](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-A56453AE-715F-489E-B0E1-B0A9EC81D33D)** on B, and B is a **[referenced object](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-031F5F29-8216-43EB-83C5-D4AF79CE540E)** for A.

Oracle Database provides an automatic mechanism to ensure that a dependent object is always up to date with respect to its referenced objects. When you create a dependent object, the database tracks dependencies between the dependent object and its referenced objects. When a referenced object changes in a way that might affect a dependent object, the database marks the dependent object invalid. For example, if a user drops a table, no view based on the dropped table is usable.

An invalid dependent object must be recompiled against the new definition of a referenced object before the dependent object is usable. Recompilation occurs automatically when the invalid dependent object is referenced.

As an illustration of how schema objects can create dependencies, the following sample script creates a table `test_table` and then a procedure that queries this table:

```
CREATE TABLE test_table ( col1 INTEGER, col2 INTEGER );

CREATE OR REPLACE PROCEDURE test_proc
AS
BEGIN
 FOR x IN ( SELECT col1, col2 FROM test_table )
 LOOP
   -- process data
   NULL;
 END LOOP;
END;
/
```

The following query of the status of procedure `test_proc` shows that it is valid:

```
SQL> SELECT OBJECT_NAME, STATUS FROM USER_OBJECTS WHERE OBJECT_NAME = 'TEST_PROC';
 
OBJECT_NAME STATUS
----------- -------
TEST_PROC   VALID
```

After adding the `col3` column to `test_table`, the procedure is still valid because the procedure has no dependencies on this column:

```
SQL> ALTER TABLE test_table ADD col3 NUMBER;
 
Table altered.
 
SQL> SELECT OBJECT_NAME, STATUS FROM USER_OBJECTS WHERE OBJECT_NAME = 'TEST_PROC';
 
OBJECT_NAME STATUS
----------- -------
TEST_PROC   VALID
```

However, changing the data type of the `col1` column, which the `test_proc` procedure depends on in, invalidates the procedure:

```
SQL> ALTER TABLE test_table MODIFY col1 VARCHAR2(20);
 
Table altered.
 
SQL> SELECT OBJECT_NAME, STATUS FROM USER_OBJECTS WHERE OBJECT_NAME = 'TEST_PROC';
 
OBJECT_NAME STATUS
----------- -------
TEST_PROC   INVALID
```

Running or recompiling the procedure makes it valid again, as shown in the following example:

```
SQL> EXECUTE test_proc
 
PL/SQL procedure successfully completed.
 
SQL> SELECT OBJECT_NAME, STATUS FROM USER_OBJECTS WHERE OBJECT_NAME = 'TEST_PROC';
 
OBJECT_NAME STATUS
----------- -------
TEST_PROC   VALID
```

## SYS and SYSTEM Schemas

All Oracle databases include default administrative accounts. Administrative accounts are highly privileged and are intended only for DBAs authorized to perform tasks such as starting and stopping the database, managing memory and storage, creating and managing database users, and so on.

The `SYS` administrative account is automatically created when a database is created. This account can perform all database administrative functions. The `SYS` schema stores the base tables and views for the **[data dictionary](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-D6A3934F-BA6D-464F-9612-C683E20514A4)**. These base tables and views are critical for the operation of Oracle Database. Tables in the `SYS` schema are manipulated only by the database and must never be modified by any user.

The `SYSTEM` administrative account is also automatically created when a database is created. The `SYSTEM` schema stores additional tables and views that display administrative information, and internal tables and views used by various Oracle Database options and tools. Never use the `SYSTEM` schema to store tables of interest to nonadministrative users.

## Sample Schemas

An Oracle database may include **sample schemas**, which are a set of interlinked schemas that enable Oracle documentation and Oracle instructional materials to illustrate common database tasks.

The `hr` sample schema contains information about employees, departments and locations, work histories, and so on. The following illustration depicts an entity-relationship diagram of the tables in `hr`. Most examples in this manual use objects from this schema.

***Figure 2-3 HR Schema***
![Figure 2-3 HR Schema](https://docs.oracle.com/database/121/CNCPT/img/GUID-5D6B55DA-2A66-41C8-999A-A2BC4919175F-default.gif)

## Overview of Tables

A **table** is the basic unit of data organization in an Oracle database.

A table describes an **entity**, which is something of significance about which information must be recorded. For example, an employee could be an entity.

Oracle Database tables fall into the following basic categories:

* Relational tables

  Relational tables have simple columns and are the most common table type. Example 2-1 shows a `CREATE TABLE` statement for a relational table.

* Object tables

  The columns correspond to the top-level attributes of an **[object type](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-33C40B41-75EF-4B5A-AE9F-45CB91700218)**. See "[Overview of Object Tables](https://docs.oracle.com/database/121/CNCPT/tablecls.htm#GUID-EBB02ED3-525E-4D7F-B984-E8AC3BF039F7)".

You can create a relational table with the following organizational characteristics:

* A **[heap-organized table](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-861D8FCE-B86C-46B3-AA01-35066D24F4CF)** does not store rows in any particular order. The `CREATE TABLE` statement creates a heap-organized table by default.


* An **[index-organized table](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-FD628CF1-3A67-4171-92BB-D4A74119ACD9)** orders rows according to the primary key values. For some applications, index-organized tables enhance performance and use disk space more efficiently. See "[Overview of Index-Organized Tables](https://docs.oracle.com/database/121/CNCPT/indexiot.htm#GUID-DAEC075B-C16D-4A57-898C-70EBCB364F0C)".


* An **[external table](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-1B18B978-29DD-4961-BD39-86838CB5F7D0)** is a read-only table whose metadata is stored in the database but whose data is stored outside the database. See "[Overview of External Tables](https://docs.oracle.com/database/121/CNCPT/tablecls.htm#GUID-97FC9DFF-A6CF-46CD-9F6F-D88A37C0E79C)".


A table is either permanent or temporary. A permanent table definition and data persist across sessions. A **[temporary table](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-41501962-A37B-4D37-BAF2-59030048FE08)** definition persists in the same way as a permanent table definition, but the data exists only for the duration of a **[transaction](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-212D8EA1-D704-4D7B-A72D-72001965CE45)** or **[session](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-0F44C072-9841-4E2E-B846-FB16A2E54139)**. Temporary tables are useful in applications where a result set must be held temporarily, perhaps because the result is constructed by running multiple operations.

### Columns

A table definition includes a table name and set of columns.

A **[column](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-F1D9AA5F-C66A-4D5F-A84C-8D7360DE3BE5)** identifies an attribute of the entity described by the table. For example, the column `employee_id` in the `employees` table refers to the employee ID attribute of an employee entity.

In general, you give each column a column name, a **[data type](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-3873B26C-657D-4508-B13D-9155F1D5D8F4)**, and a width when you create a table. For example, the data type for `employee_id` is `NUMBER(6)`, indicating that this column can only contain numeric data up to 6 digits in width. The width can be predetermined by the data type, as with `DATE`.

#### Virtual Columns

A table can contain a **virtual column**, which unlike a nonvirtual column does not consume disk space.

The database derives the values in a virtual column on demand by computing a set of user-specified expressions or functions. For example, the virtual column `income` could be a function of the `salary` and `commission_pct` columns.

#### Invisible Columns

An **invisible column** is a user-specified column whose values are only visible when the column is explicitly specified by name. You can add an invisible column to a table without affecting existing applications, and make the column visible if necessary.

In general, invisible columns help migrate and evolve online applications. A use case might be an application that queries a three-column table with a `SELECT *` statement. Adding a fourth column to the table would break the application, which expects three columns of data. Adding a fourth invisible column makes the application function normally. A developer can then alter the application to handle a fourth column, and make the column visible when the application goes live.

The following example creates a table `products` with an invisible column `count`, and then makes the invisible column visible:

`
CREATE TABLE products ( prod_id INT, count INT INVISIBLE );
ALTER TABLE products MODIFY ( count VISIBLE );
`

### Rows

A **row** is a collection of column information corresponding to a record in a table.

For example, a row in the `employees` table describes the attributes of a specific employee: employee ID, last name, first name, and so on. After you create a table, you can insert, query, delete, and update rows using SQL.

# Example: CREATE TABLE and ALTER TABLE Statements

The Oracle SQL statement to create a table is `CREATE TABLE`.

***Example 2-1 CREATE TABLE employees***

The following example shows the `CREATE TABLE` statement for the `employees` table in the `hr` sample schema. The statement specifies columns such as `employee_id`, `first_name`, and so on, specifying a data type such as `NUMBER` or `DATE` for each column.

`
CREATE TABLE employees
    ( employee_id    NUMBER(6)
    , first_name     VARCHAR2(20)
    , last_name      VARCHAR2(25)
         CONSTRAINT     emp_last_name_nn  NOT NULL
    , email          VARCHAR2(25)
        CONSTRAINT     emp_email_nn  NOT NULL
    , phone_number   VARCHAR2(20)
    , hire_date      DATE
        CONSTRAINT     emp_hire_date_nn  NOT NULL
    , job_id         VARCHAR2(10)
        CONSTRAINT     emp_job_nn  NOT NULL
    , salary         NUMBER(8,2)
    , commission_pct NUMBER(2,2)
    , manager_id     NUMBER(6)
    , department_id  NUMBER(4)
    , CONSTRAINT     emp_salary_min
                     CHECK (salary > 0)
    , CONSTRAINT     emp_email_uk
                     UNIQUE (email)
    ) ;
`

***Example 2-2 ALTER TABLE employees***

The following example shows an `ALTER TABLE` statement that adds integrity constraints to the `employees` table. Integrity constraints enforce business rules and prevent the entry of invalid information into tables.

`
ALTER TABLE employees
ADD ( CONSTRAINT     emp_emp_id_pk
                       PRIMARY KEY (employee_id)
    , CONSTRAINT     emp_dept_fk
                       FOREIGN KEY (department_id)
                         REFERENCES departments
    , CONSTRAINT     emp_job_fk
                       FOREIGN KEY (job_id)
                         REFERENCES jobs (job_id)
    , CONSTRAINT     emp_manager_fk
                       FOREIGN KEY (manager_id)
                         REFERENCES employees
    ) ;
`

***Example 2-3 Rows in the employees Table***

The following sample output shows 8 rows and 6 columns of the `hr.employees` table.
```
EMPLOYEE_ID FIRST_NAME  LAST_NAME      SALARY COMMISSION_PCT DEPARTMENT_ID
----------- ----------- ------------- ------- -------------- -------------
        100 Steven      King            24000                           90
        101 Neena       Kochhar         17000                           90
        102 Lex         De Haan         17000                           90
        103 Alexander   Hunold           9000                           60
        107 Diana       Lorentz          4200                           60
        149 Eleni       Zlotkey         10500             .2            80
        174 Ellen       Abel            11000             .3            80
        178 Kimberely   Grant            7000            .15
```

The preceding output illustrates some of the following important characteristics of tables, columns, and rows:

* A row of the table describes the attributes of one employee: name, salary, department, and so on. For example, the first row in the output shows the record for the employee named Steven King.


* A column describes an attribute of the employee. In the example, the `employee_id` column is the **[primary key](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-8640EFA5-276C-4812-A078-1F21F55F4200)**, which means that every employee is uniquely identified by employee ID. Any two employees are guaranteed not to have the same employee ID.


* A non-key column can contain rows with identical values. In the example, the salary value for employees 101 and 102 is the same: `17000`.


* A **[foreign key](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-EEE8A67D-6B0E-468E-9554-48FEE552BC9A)** column refers to a primary or unique key in the same table or a different table. In this example, the value of `90` in `department_id` corresponds to the `department_id` column of the `departments` table.


* A **[field](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-CCCDD0D5-7738-431A-96F4-08B32D01FD7B)** is the intersection of a row and column. It can contain only one value. For example, the field for the department ID of employee 103 contains the value `60`.


* A field can lack a value. In this case, the field is said to contain a null value. The value of the `commission_pct` column for employee 100 is null, whereas the value in the field for employee 149 is `.2`. A column allows nulls unless a `NOT NULL` or primary key integrity constraint has been defined on this column, in which case no row can be inserted without a value for this column.

### Oracle Data Types

Each column has a **data type**, which is associated with a specific storage format, constraints, and valid range of values. The data type of a value associates a fixed set of properties with the value.

These properties cause Oracle Database to treat values of one data type differently from values of another. For example, you can multiply values of the `NUMBER` data type, but not values of the `RAW` data type.

When you create a table, you must specify a data type for each of its columns. Each value subsequently inserted in a column assumes the column data type.

Oracle Database provides several built-in data types. The most commonly used data types fall into the following categories:

* Character Data Types

* Numeric Data Types

* Datetime Data Types

* Rowid Data Types

* Format Models and Data Types

Other important categories of built-in types include raw, large objects (LOBs), and collections. PL/SQL has data types for constants and variables, which include `BOOLEAN`, reference types, composite types (records), and user-defined types.

#### Character Data Types

Character data types store alphanumeric data in strings. The most common character data type is `VARCHAR2`, which is the most efficient option for storing character data.

The byte values correspond to the **[character encoding](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-770C0F15-8AA7-446B-9DF8-455B8B406A4D)** scheme, generally called a **[character set](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-9EDC68FA-88D3-4F15-85C4-2B603C3240CE)**. The database character set is established at database creation. Examples of character sets are 7-bit ASCII, EBCDIC, and Unicode UTF-8.

The length semantics of character data types are measurable in bytes or characters. The treatment of strings as a sequence of bytes is called **[byte semantics](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-BBEA62B4-5767-40DF-B2D6-BEC1F267B22F)**. This is the default for character data types. The treatment of strings as a sequence of characters is called **[character semantics](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-46D094F4-88B8-4939-AFBD-971EA3AABBDA)**. A character is a code point of the database character set.

#### VARCHAR2 and CHAR Data Types
The `VARCHAR2` data type stores variable-length character literals. A **literal** is a fixed data value.

For example, `'LILA', 'St. George Island'`, and `'101'` are all character literals; `5001` is a numeric literal. Character literals are enclosed in single quotation marks so that the database can distinguish them from schema object names.

When you create a table with a `VARCHAR2` column, you specify a maximum string length. In Example 2-1, the `last_name` column has a data type of `VARCHAR2(25)`, which means that any name stored in the column has a maximum of 25 bytes.

For each row, Oracle Database stores each value in the column as a variable-length field unless a value exceeds the maximum length, in which case the database returns an error. For example, in a single-byte character set, if you enter 10 characters for the `last_name` column value in a row, then the column in the row piece stores only 10 characters (10 bytes), not 25. Using `VARCHAR2` reduces space consumption.

In contrast to `VARCHAR2`, `CHAR` stores fixed-length character strings. When you create a table with a `CHAR` column, the column requires a string length. The default is 1 byte. The database uses blanks to pad the value to the specified length.

Oracle Database compares `VARCHAR2` values using nonpadded comparison semantics and compares `CHAR` values using blank-padded comparison semantics.

#### NCHAR and NVARCHAR2 Data Types
The `NCHAR` and `NVARCHAR2` data types store Unicode character data.

**Unicode** is a universal encoded character set that can store information in any language using a single character set. `NCHAR` stores fixed-length character strings that correspond to the national character set, whereas `NVARCHAR2` stores variable length character strings.

You specify a national character set when creating a database. The character set of `NCHAR` and `NVARCHAR2` data types must be either `AL16UTF16` or `UTF8`. Both character sets use Unicode encoding.

When you create a table with an `NCHAR` or `NVARCHAR2` column, the maximum size is always in character length semantics. Character length semantics is the default and only length semantics for `NCHAR` or `NVARCHAR2`.

### Numeric Data Types

The Oracle Database numeric data types store fixed and floating-point numbers, zero, and infinity. Some numeric types also store values that are the undefined result of an operation, which is known as "not a number" or `NaN`.

Oracle Database stores numeric data in variable-length format. Each value is stored in scientific notation, with 1 byte used to store the exponent. The database uses up to 20 bytes to store the **[mantissa](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-0912AA8C-6F9E-4382-97A3-2B43832BEBE6)**, which is the part of a floating-point number that contains its significant digits. Oracle Database does not store leading and trailing zeros.

### NUMBER Data Type

The `NUMBER` data type stores fixed and floating-point numbers. The database can store numbers of virtually any magnitude. This data is guaranteed to be portable among different operating systems running Oracle Database. The `NUMBER` data type is recommended for most cases in which you must store numeric data.

You specify a fixed-point number in the form `NUMBER(p,s)`, where *`p`* and *`s`* refer to the following characteristics:

* Precision

  The **[precision](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-84EF5156-C5F0-49CA-BFAD-AAEC3C5F9289)** specifies the total number of digits. If a precision is not specified, then the column stores the values exactly as provided by the application without any rounding.


* Scale

  The **[scale](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-7CA2255F-476D-461C-967A-6FABC5DAA8CE)** specifies the number of digits from the decimal point to the least significant digit. Positive scale counts digits to the right of the decimal point up to and including the least significant digit. Negative scale counts digits to the left of the decimal point up to but not including the least significant digit. If you specify a precision without a scale, as in `NUMBER(6)`, then the scale is 0.

In Example 2-1, the `salary` column is type `NUMBER(8,2)`, so the precision is 8 and the scale is 2. Thus, the database stores a salary of 100,000 as 100000.00.

#### Floating-Point Numbers

Oracle Database provides two numeric data types exclusively for floating-point numbers: `BINARY_FLOAT` and `BINARY_DOUBLE`.

These types support all of the basic functionality provided by the `NUMBER` data type. However, whereas `NUMBER` uses decimal precision, `BINARY_FLOAT` and `BINARY_DOUBLE` use binary precision, which enables faster arithmetic calculations and usually reduces storage requirements.

`BINARY_FLOAT` and `BINARY_DOUBLE` are approximate numeric data types. They store approximate representations of decimal values, rather than exact representations. For example, the value 0.1 cannot be exactly represented by either `BINARY_DOUBLE` or `BINARY_FLOAT`. They are frequently used for scientific computations. Their behavior is similar to the data types `FLOAT` and `DOUBLE` in Java and XMLSchema.

### Datetime Data Types

The **datetime** data types are `DATE` and `TIMESTAMP`. Oracle Database provides comprehensive time zone support for time stamps.

#### DATE Data Type

The `DATE` data type stores date and time. Although datetimes can be represented in character or number data types, `DATE` has special associated properties.

The database stores dates internally as numbers. Dates are stored in fixed-length fields of 7 bytes each, corresponding to century, year, month, day, hour, minute, and second.

The database displays dates according to the specified **[format model](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-F97AA700-3D17-428F-B9E0-52259D19C037)**. A format model is a character literal that describes the format of a datetime in a character string. The standard date format is `DD-MON-RR`, which displays dates in the form `01-JAN-11`.

`RR` is similar to `YY` (the last two digits of the year), but the century of the return value varies according to the specified two-digit year and the last two digits of the current year. Assume that in 1999 the database displays `01-JAN-11`. If the date format uses `RR`, then `11` specifies `2011`, whereas if the format uses `YY`, then `11` specifies `1911`. You can change the default date format at both the database instance and session level.

Oracle Database stores time in 24-hour format—`HH:MI:SS`. If no time portion is entered, then by default the time in a date field is `00:00:00 A.M.` In a time-only entry, the date portion defaults to the first day of the current month.

#### TIMESTAMP Data Type

The `TIMESTAMP` data type is an extension of the `DATE` data type.

`TIMESTAMP` stores fractional seconds in addition to the information stored in the `DATE` data type. The `TIMESTAMP` data type is useful for storing precise time values, such as in applications that must track event order.

The `DATETIME` data types `TIMESTAMP WITH TIME ZONE` and `TIMESTAMP WITH LOCAL TIME ZONE` are time-zone aware. When a user selects the data, the value is adjusted to the time zone of the user session. This data type is useful for collecting and evaluating date information across geographic regions.

### Rowid Data Types

Every row stored in the database has an address. Oracle Database uses a `ROWID` data type to store the address (rowid) of every row in the database.

Rowids fall into the following categories:

* Physical rowids store the addresses of rows in heap-organized tables, table clusters, and table and index partitions.

* Logical rowids store the addresses of rows in index-organized tables.

* Foreign rowids are identifiers in foreign tables, such as DB2 tables accessed through a gateway. They are not standard Oracle Database rowids.

A data type called the **[universal rowid](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-288B0853-9B77-4A7F-AEE9-8924EA7877A7)**, or urowid, supports all types of rowids.

#### Use of Rowids
Oracle Database uses rowids internally for the construction of indexes.

A **[B-tree index](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-8D6D0C64-6AC8-4B22-A9AF-1B62F61AE10B)**, which is the most common type, contains an ordered list of keys divided into ranges. Each key is associated with a rowid that points to the associated row's address for fast access.

End users and application developers can also use rowids for several important functions:

* Rowids are the fastest means of accessing particular rows.

* Rowids provide the ability to see how a table is organized.

* Rowids are unique identifiers for rows in a given table.

You can also create tables with columns defined using the `ROWID` data type. For example, you can define an exception table with a column of data type `ROWID` to store the rowids of rows that violate integrity constraints. Columns defined using the `ROWID` data type behave like other table columns: values can be updated, and so on.

#### ROWID Pseudocolumn

Every table in an Oracle database has a pseudocolumn named `ROWID`.

A **[pseudocolumn](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-175D4923-5C7E-4FF0-A69B-C4D8F3D93A3D)** behaves like a table column, but is not actually stored in the table. You can select from pseudocolumns, but you cannot insert, update, or delete their values. A pseudocolumn is also similar to a SQL **[function](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-5611828A-3621-4527-AEB1-12017A454E94)** without arguments. Functions without arguments typically return the same value for every row in the result set, whereas pseudocolumns typically return a different value for each row.

Values of the `ROWID` pseudocolumn are strings representing the address of each row. These strings have the data type `ROWID`. This pseudocolumn is not evident when listing the structure of a table by executing `SELECT` or `DESCRIBE`, nor does the pseudocolumn consume space. However, the rowid of each row can be retrieved with a SQL query using the reserved word `ROWID` as a column name.

The following example queries the `ROWID` pseudocolumn to show the rowid of the row in the `employees` table for employee 100:

```
SQL> SELECT ROWID FROM employees WHERE employee_id = 100;

ROWID
------------------
AAAPecAAFAAAABSAAA
```

### Format Models and Data Types

A **format model** is a character literal that describes the format of datetime or numeric data stored in a character string. A format model does not change the internal representation of the value in the database.

When you convert a character string into a date or number, a format model determines how the database interprets the string. In SQL, you can use a format model as an argument of the `TO_CHAR` and `TO_DATE` functions to format a value to be returned from the database or to format a value to be stored in the database.

The following statement selects the salaries of the employees in Department 80 and uses the `TO_CHAR` function to convert these salaries into character values with the format specified by the number format model `'$99,990.99'`:

```
SQL> SELECT last_name employee, TO_CHAR(salary, '$99,990.99') AS "SALARY"
  2  FROM   employees
  3  WHERE  department_id = 80 AND last_name = 'Russell';
 
EMPLOYEE                  SALARY
------------------------- -----------
Russell                    $14,000.00
```

The following example updates a hire date using the `TO_DATE` function with the format mask `'YYYY MM DD'` to convert the string `'1998 05 20'` to a `DATE` value:

```
SQL> UPDATE employees
  2  SET hire_date = TO_DATE('1998 05 20','YYYY MM DD')
  3  WHERE last_name = 'Hunold';
```

## Integrity Constraints

An **integrity constraint** is a named rule that restrict the values for one or more columns in a table. These rules prevent invalid data entry into tables. Also, constraints can prevent the deletion of a table when certain dependencies exist.

If a constraint is enabled, then the database checks data as it is entered or updated. Oracle Database prevents data that does not conform to the constraint from being entered. If a constraint is disabled, then Oracle Database allows data that does not conform to the constraint to enter the database.

In Example 2-1, the `CREATE TABLE` statement specifies `NOT NULL` constraints for the `last_name`, `email`, `hire_date`, and `job_id` columns. The constraint clauses identify the columns and the conditions of the constraint. These constraints ensure that the specified columns contain no null values. For example, an attempt to insert a new employee without a job ID generates an error.

You can create a constraint when or after you create a table. You can temporarily disable constraints if needed. The database stores constraints in the **[data dictionary](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-D6A3934F-BA6D-464F-9612-C683E20514A4)**.

## Table Storage

Oracle Database uses a **data segment** in a tablespace to hold table data.

A segment contains extents made up of data blocks. The data segment for a table (or cluster data segment, for a **[table cluster](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-1C56177E-6BEE-4FE7-B45E-38298CDB946D)** is located in either the default tablespace of the table owner or in a tablespace named in the `CREATE TABLE` statement.

### Table Organization

By default, a table is organized as a heap, which means that the database places rows where they fit best rather than in a user-specified order. Thus, a heap-organized table is an unordered collection of rows.

As users add rows, the database places the rows in the first available free space in the data segment. Rows are not guaranteed to be retrieved in the order in which they were inserted.

The `hr.departments` table is a heap-organized table. It has columns for department ID, name, manager ID, and location ID. As rows are inserted, the database stores them wherever they fit. A data block in the table segment might contain the unordered rows shown in the following example:

```
50,Shipping,121,1500
120,Treasury,,1700
70,Public Relations,204,2700
30,Purchasing,114,1700
130,Corporate Tax,,1700
10,Administration,200,1700
110,Accounting,205,1700
```

The column order is the same for all rows in a table. The database usually stores columns in the order in which they were listed in the `CREATE TABLE` statement, but this order is not guaranteed. For example, if a table has a column of type `LONG`, then Oracle Database always stores this column last in the row. Also, if you add a new column to a table, then the new column becomes the last column stored.

A table can contain a **[virtual column](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-E95FC6AD-C932-4DE2-9D7B-B98D1168E7DA)**, which unlike normal columns does not consume space on disk. The database derives the values in a virtual column on demand by computing a set of user-specified expressions or functions. You can index virtual columns, collect statistics on them, and create integrity constraints. Thus, virtual columns are much like nonvirtual columns.

### Row Storage

The database stores rows in data blocks. Each row of a table containing data for less than 256 columns is contained in one or more row pieces.

If possible, Oracle Database stores each row as one **[row piece](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-4C16E45D-79AF-40D5-ACE2-CE2B301CA97B)**. However, if all of the row data cannot be inserted into a single data block, or if an update to an existing row causes the row to outgrow its data block, then the database stores the row using multiple row pieces (see "[Data Block Format](https://docs.oracle.com/database/121/CNCPT/logical.htm#GUID-754ECC03-DD58-4B49-95D1-B98A23B508B2)").

Rows in a table cluster contain the same information as rows in nonclustered tables. Additionally, rows in a table cluster contain information that references the cluster key to which they belong.


### Rowids of Row Pieces
A **rowid** is effectively a 10-byte physical address of a row.

Every row in a heap-organized table has a rowid unique to this table that corresponds to the physical address of a row piece. For table clusters, rows in different tables that are in the same data block can have the same rowid.

Oracle Database uses rowids internally for the construction of indexes. For example, each key in a **[B-tree index](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-8D6D0C64-6AC8-4B22-A9AF-1B62F61AE10B)** is associated with a rowid that points to the address of the associated row for fast access (see "[Overview of B-Tree Indexes](https://docs.oracle.com/database/121/CNCPT/indexiot.htm#GUID-FC93A85B-C237-4249-AD1E-FF54576ED050)"). Physical rowids provide the fastest possible access to a table row, enabling the database to retrieve a row in as little as a single I/O.

### Storage of Null Values

A **null** is the absence of a value in a column. Nulls indicate missing, unknown, or inapplicable data.

Nulls are stored in the database if they fall between columns with data values. In these cases, they require 1 byte to store the length of the column (zero). Trailing nulls in a row require no storage because a new row header signals that the remaining columns in the previous row are null. For example, if the last three columns of a table are null, then no data is stored for these columns.

### Table Compression

The database can use **table compression** to reduce the amount of storage required for the table.

Compression saves disk space, reduces memory use in the database buffer cache, and in some cases speeds query execution. Table compression is transparent to database applications.

#### Basic Table Compression and Advanced Row Compression

Dictionary-based table compression provides good compression ratios for heap-organized tables.

Oracle Database supports the following types of dictionary-based table compression:

* Basic table compression

  This type of compression is intended for bulk load operations. The database does not compress data modified using conventional DML. You must use **[direct path INSERT](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-73D6FDFE-E38D-49F2-A5A5-B6BDB2FD297C)** operations, `ALTER TABLE . . . MOVE` operations, or online table redefinition to achieve basic table compression.
  

* Advanced row compression

  This type of compression is intended for OLTP applications and compresses data manipulated by any SQL operation. The database achieves a competitive compression ratio while enabling the application to perform DML in approximately the same amount of time as DML on an uncompressed table.

For the preceding types of compression, the database stores compressed rows in **[row major format](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-68A36A9F-BD3E-4245-909F-D68B3D056352)**. All columns of one row are stored together, followed by all columns of the next row, and so on (see "[Row Format](https://docs.oracle.com/database/121/CNCPT/logical.htm#GUID-D993A749-93D0-448C-BDC8-330D805AC481)"). The database replaces duplicate values with a short reference to a symbol table stored at the beginning of the block. Thus, information that the database needs to re-create the uncompressed data is stored in the data block itself.

Compressed data blocks look much like normal data blocks. Most database features and functions that work on regular data blocks also work on compressed blocks.

You can declare compression at the tablespace, table, partition, or subpartition level. If specified at the tablespace level, then all tables created in the tablespace are compressed by default.

***Example 2-4 Table-Level Compression***

The following statement applies advanced row compression to the `orders` table:

`ALTER TABLE oe.orders ROW STORE COMPRESS ADVANCED;`

***Example 2-5 Partition-Level Compression***

The following example of a partial `CREATE TABLE` statement specifies advanced row compression for one partition and basic table compression for the other partition:

```
CREATE TABLE sales (
    prod_id     NUMBER     NOT NULL,
    cust_id     NUMBER     NOT NULL, ... )
 PCTFREE 5 NOLOGGING NOCOMPRESS
 PARTITION BY RANGE (time_id)
 ( partition sales_2013 VALUES LESS THAN(TO_DATE(...)) ROW STORE COMPRESS BASIC,
   partition sales_2014 VALUES LESS THAN (MAXVALUE) ROW STORE COMPRESS ADVANCED );
```

### Hybrid Columnar Compression

With Hybrid Columnar Compression, the database stores the same column for a group of rows together. The data block does not store data in row-major format, but uses a combination of both row and columnar methods.

Storing column data together, with the same data type and similar characteristics, dramatically increases the storage savings achieved from compression. The database compresses data manipulated by any SQL operation, although compression levels are higher for direct path loads. Database operations work transparently against compressed objects, so no application changes are required.

### Types of Hybrid Columnar Compression

If your underlying storage supports Hybrid Columnar Compression, then you can specify different types of compression, depending on your requirements.

The compression options are:

* Warehouse compression

  This type of compression is optimized to save storage space, and is intended for data warehouse applications.


* Online archival compression

    This type of compression is optimized for maximum compression levels, and is intended for historical data and data that does not change.

To achieve warehouse or online archival compression, you must use direct path loads, `ALTER TABLE . . . MOVE` operations, or online table redefinition.

Hybrid Columnar Compression is optimized for Data Warehousing and decision support applications on Oracle Exadata storage. Oracle Exadata maximizes the performance of queries on tables that are compressed using Hybrid Columnar Compression, taking advantage of the processing power, memory, and Infiniband network bandwidth that are integral to the Oracle Exadata storage server.

Other Oracle storage systems support Hybrid Columnar Compression, and deliver the same space savings as on Oracle Exadata storage, but do not deliver the same level of query performance. For these storage systems, Hybrid Columnar Compression is ideal for in-database archiving of older data that is infrequently accessed.

### Compression Units

Hybrid Columnar Compression uses a logical construct called a **compression unit** to store a set of rows.

When you load data into a table, the database stores groups of rows in columnar format, with the values for each column stored and compressed together. After the database has compressed the column data for a set of rows, the database fits the data into the compression unit.

For example, you apply Hybrid Columnar Compression to a `daily_sales` table. At the end of every day, you populate the table with items and the number sold, with the item ID and date forming a composite primary key. Table 2-1 shows a subset of the rows in `daily_sales`.

***Table 2-1 Sample Table daily_sales***

| Item_ID | Date      | Num_Sold | Shipped_From | Restock |
|:-------:|:---------:|:--------:|:------------:|:-------:|
| 1000    | 01-JUN-14 | 2        | WAREHOUSE1   | Y       |
| 1001    | 01-JUN-14 | 0        | WAREHOUSE3   | N       |
| 1002    | 01-JUN-14 | 1        | WAREHOUSE3   | N       |
| 1003    | 01-JUN-14 | 0        | WAREHOUSE2   | N       |
| 1004    | 01-JUN-14 | 2        | WAREHOUSE1   | N       |
| 1005    | 01-JUN-14 | 1        | WAREHOUSE2   | N       |

Assume that the rows in **Table 2-1** are stored in one compression unit. Hybrid Columnar Compression stores the values for each column together, and then uses multiple algorithms to compress each column. The database chooses the algorithms based on a variety of factors, including the data type of the column, the cardinality of the actual values in the column, and the compression level chosen by the user.

As shown in Figure 2-4, each compression unit can span multiple data blocks. The values for a particular column may or may not span multiple blocks.

***Figure 2-4 Compression Unit***
![Figure 2-4](https://docs.oracle.com/database/121/CNCPT/img/GUID-8A3B7541-A1A3-4250-9943-B94C02BCDAF5-default.gif)

Hybrid Columnar Compression has implications for row locking (see "[Row Locks (TX)](https://docs.oracle.com/database/121/CNCPT/consist.htm#GUID-92D38952-A7A0-486F-9547-BDB6E624F125)"). When an update occurs for a row in an uncompressed data block, only the updated row is locked. In contrast, the database must lock all rows in the compression unit if an update is made to any row in the unit. Updates to rows using Hybrid Columnar Compression cause rowids to change.

## Overview of Table Clusters

A **table cluster** is a group of tables that share common columns and store related data in the same blocks.

When tables are clustered, a single data block can contain rows from multiple tables. For example, a block can store rows from both the `employees` and `departments` tables rather than from only a single table.

The **[cluster key](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-0721296C-D83A-4EA4-80F2-44AC3F1E9112)** is the column or columns that the clustered tables have in common. For example, the `employees` and `departments` tables share the `department_id` column. You specify the cluster key when creating the table cluster and when creating every table added to the table cluster.

The cluster key value is the value of the cluster key columns for a particular set of rows. All data that contains the same cluster key value, such as `department_id=20`, is physically stored together. Each cluster key value is stored only once in the cluster and the cluster index, no matter how many rows of different tables contain the value.

For an analogy, suppose an HR manager has two book cases: one with boxes of employee folders and the other with boxes of department folders. Users often ask for the folders for all employees in a particular department. To make retrieval easier, the manager rearranges all the boxes in a single book case. She divides the boxes by department ID. Thus, all folders for employees in department 20 and the folder for department 20 itself are in one box; the folders for employees in department 100 and the folder for department 100 are in another box, and so on.

Consider clustering tables when they are primarily queried (but not modified) and records from the tables are frequently queried together or joined. Because table clusters store related rows of different tables in the same data blocks, properly used table clusters offer the following benefits over nonclustered tables:

* Disk I/O is reduced for joins of clustered tables.


* Access time improves for joins of clustered tables.


* Less storage is required to store related table and index data because the cluster key value is not stored repeatedly for each row.


Typically, clustering tables is not appropriate in the following situations:

* The tables are frequently updated.


* The tables frequently require a **[full table scan](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-BF9B54D6-892E-4C3B-8536-38958ACC069D)**.


* The tables require truncating.

### Overview of Indexed Clusters

An **index cluster** is a table cluster that uses an index to locate data. The **cluster index** is a B-tree index on the cluster key. A cluster index must be created before any rows can be inserted into clustered tables.

***Example 2-6 Creating a Table Cluster and Associated Index***

Assume that you create the cluster `employees_departments_cluster` with the cluster key `department_id`, as shown in the following example:

```
CREATE CLUSTER employees_departments_cluster
   (department_id NUMBER(4))
SIZE 512;

CREATE INDEX idx_emp_dept_cluster 
   ON CLUSTER employees_departments_cluster;
```

Because the `HASHKEYS` clause is not specified, `employees_departments_cluster` is an indexed cluster. The preceding example creates an index named `idx_emp_dept_cluster` on the cluster key `department_id`.

***Example 2-7 Creating Tables in an Indexed Cluster***

You create the `employees` and `departments` tables in the cluster, specifying the `department_id` column as the cluster key, as follows (the ellipses mark the place where the column specification goes):

```
CREATE TABLE employees ( ... )
   CLUSTER employees_departments_cluster (department_id);
 
CREATE TABLE departments ( ... )
   CLUSTER employees_departments_cluster (department_id);
```

Assume that you add rows to the `employees` and `departments` tables. The database physically stores all rows for each department from the `employees` and `departments` tables in the same data blocks. The database stores the rows in a heap and locates them with the index.

Figure 2-5 shows the `employees_departments_cluster` table cluster, which contains `employees` and `departments`. The database stores rows for employees in department 20 together, department 110 together, and so on. If the tables are not clustered, then the database does not ensure that the related rows are stored together.

***Figure 2-5 Clustered Table Data***
![Figure 2-5](https://docs.oracle.com/database/121/CNCPT/img/GUID-F32585AD-6A0D-4AE1-9585-7D21A4505143-default.gif)

The B-tree cluster index associates the cluster key value with the database block address (DBA) of the block containing the data. For example, the index entry for key 20 shows the address of the block that contains data for employees in department 20:

`20,AADAAAA9d`

The cluster index is separately managed, just like an index on a nonclustered table, and can exist in a separate tablespace from the table cluster.

### Overview of Hash Clusters

A **hash cluster** is like an indexed cluster, except the index key is replaced with a **hash function**. No separate cluster index exists. In a hash cluster, the data is the index.

With an indexed table or indexed cluster, Oracle Database locates table rows using key values stored in a separate index. To find or store a row in an indexed table or table cluster, the database must perform at least two I/Os:

* One or more I/Os to find or store the key value in the index


* Another I/O to read or write the row in the table or table cluster


To find or store a row in a hash cluster, Oracle Database applies the hash function to the cluster key value of the row. The resulting hash value corresponds to a data block in the cluster, which the database reads or writes on behalf of the issued statement.

Hashing is an optional way of storing table data to improve the performance of data retrieval. Hash clusters may be beneficial when the following conditions are met:

* A table is queried much more often than modified.


* The hash key column is queried frequently with equality conditions, for example, `WHERE department_id=20`. For such queries, the cluster key value is hashed. The hash key value points directly to the disk area that stores the rows.


* You can reasonably guess the number of hash keys and the size of the data stored with each key value.

#### Hash Cluster Creation

To create a hash cluster, you use the same `CREATE CLUSTER` statement as for an indexed cluster, with the addition of a hash key. The number of hash values for the cluster depends on the hash key.

The cluster key, like the key of an indexed cluster, is a single column or composite key shared by the tables in the cluster. A **[hash key value](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-C3BAC9AD-DE26-4C9F-B27E-B45D2401B230)** is an actual or possible value inserted into the cluster key column. For example, if the cluster key is department_id, then hash key values could be 10, 20, 30, and so on.

Oracle Database uses a hash function that accepts an infinite number of hash key values as input and sorts them into a finite number of buckets. Each bucket has a unique numeric ID known as a **[hash value](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-385E8505-A9C0-488E-9BC8-5DEEF23C0CD2)**. Each hash value maps to the database block address for the block that stores the rows corresponding to the hash key value (department 10, 20, 30, and so on).

In the following example, the number of departments that are likely to exist is 100, so `HASHKEYS` is set to `100`:

```
CREATE CLUSTER employees_departments_cluster
   (department_id NUMBER(4))
SIZE 8192 HASHKEYS 100;
```

After you create `employees_departments_cluster`, you can create the `employees` and `departments` tables in the cluster. You can then load data into the hash cluster just as in the indexed cluster described in "[Overview of Indexed Clusters](https://docs.oracle.com/database/121/CNCPT/tablecls.htm#GUID-CC31365B-83B0-4E09-A047-BF1B79AC887A)".

### Hash Cluster Queries

In queries of a hash cluster, the database determines how to hash the key values input by the user.

For example, users frequently execute queries such as the following, entering different department ID numbers for `p_id`:

```
SELECT *
FROM   employees
WHERE  department_id = :p_id;
 
SELECT * 
FROM   departments 
WHERE  department_id = :p_id;

SELECT * 
FROM   employees e, departments d 
WHERE  e.department_id = d.department_id
AND    d.department_id = :p_id;
```

If a user queries employees in `department_id=20`, then the database might hash this value to bucket 77. If a user queries employees in `department_id=10`, then the database might hash this value to bucket 15. The database uses the internally generated hash value to locate the block that contains the employee rows for the requested department.

Figure 2-6 depicts a hash cluster segment as a horizontal row of blocks. As shown in the graphic, a query can retrieve data in a single I/O.

***Figure 2-6 Retrieving Data from a Hash Cluster***
![Figure 2-6](https://docs.oracle.com/database/121/CNCPT/img/GUID-5E1644FA-57DD-4104-B17E-1339370E9A03-default.gif)

A limitation of hash clusters is the unavailability of range scans on nonindexed cluster keys (see "[Index Range Scan](https://docs.oracle.com/database/121/CNCPT/indexiot.htm#GUID-E1BD4FBF-8A6D-4EF8-8892-7B1CBFA9D5C6)"). Assume no separate index exists for the hash cluster created in [Hash Cluster Creation](https://docs.oracle.com/database/121/CNCPT/tablecls.htm#GUID-391EC9EF-A34A-4A49-A9E4-9BF5B7324E9E). A query for departments with IDs between 20 and 100 cannot use the hashing algorithm because it cannot hash every possible value between 20 and 100. Because no index exists, the database must perform a full scan.

### Hash Cluster Variations

A single-table hash cluster is an optimized version of a hash cluster that supports only one table at a time. A one-to-one mapping exists between hash keys and rows.

A single-table hash cluster can be beneficial when users require rapid access to a table by primary key. For example, users often look up an employee record in the `employees` table by `employee_id`.

A **[sorted hash cluster](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-AFC062EE-E4F1-4B7A-ADBA-02AB939B21E6)** stores the rows corresponding to each value of the hash function in such a way that the database can efficiently return them in sorted order. The database performs the optimized sort internally. For applications that always consume data in sorted order, this technique can mean faster retrieval of data. For example, an application might always sort on the `order_date` column of the `orders` table.

### Hash Cluster Storage

Oracle Database allocates space for a hash cluster differently from an indexed cluster.

In the example in [Hash Cluster Creation](https://docs.oracle.com/database/121/CNCPT/tablecls.htm#GUID-391EC9EF-A34A-4A49-A9E4-9BF5B7324E9E), `HASHKEYS` specifies the number of departments likely to exist, whereas `SIZE` specifies the size of the data associated with each department. The database computes a storage space value based on the following formula:

`HASHKEYS * SIZE / database_block_size`

Thus, if the block size is 4096 bytes in the example shown in [Hash Cluster Creation](https://docs.oracle.com/database/121/CNCPT/tablecls.htm#GUID-391EC9EF-A34A-4A49-A9E4-9BF5B7324E9E), then the database allocates at least 200 blocks to the hash cluster.

Oracle Database does not limit the number of hash key values that you can insert into the cluster. For example, even though `HASHKEYS` is `100`, nothing prevents you from inserting 200 unique departments in the `departments` table. However, the efficiency of the hash cluster retrieval diminishes when the number of hash values exceeds the number of hash keys.

To illustrate the retrieval issues, assume that block 100 in Figure 2-6 is completely full with rows for department 20. A user inserts a new department with `department_id` 43 into the `departments` table. The number of departments exceeds the `HASHKEYS` value, so the database hashes `department_id` 43 to hash value 77, which is the same hash value used for `department_id` 20. Hashing multiple input values to the same output value is called a **[hash collision](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-2B7DCF40-8C20-439C-8DFD-E8F3EDE8B8FF)**.

When users insert rows into the cluster for department 43, the database cannot store these rows in block 100, which is full. The database links block 100 to a new overflow block, say block 200, and stores the inserted rows in the new block. Both block 100 and 200 are now eligible to store data for either department. As shown in Figure 2-7, a query of either department 20 or 43 now requires *two* I/Os to retrieve the data: block 100 and its associated block 200. You can solve this problem by re-creating the cluster with a different `HASHKEYS` value.

***Figure 2-7 Retrieving Data from a Hash Cluster When a Hash Collision Occurs***
![Figure 2-7](https://docs.oracle.com/database/121/CNCPT/img/GUID-D8E8EDE6-2BB7-4AAD-BAF4-EF72B8725E1A-default.gif)

## Overview of Attribute-Clustered Tables

An **attribute-clustered table** is a heap-organized table that stores data in close proximity on disk based on user-specified clustering directives. The directives specify columns in single or multiple tables.

The directives are as follows:

* The `CLUSTERING ... BY LINEAR ORDER` directive orders data in a table according to specified columns.

  Consider using `BY LINEAR ORDER` clustering, which is the default, when queries qualify the prefix of columns specified in the clustering clause. For example, if queries of `sh.sales` often specify either a customer ID or both customer ID and product ID, then you could cluster data in the table using the linear column order `cust_id, prod_id`.


* The `CLUSTERING ... BY INTERLEAVED ORDER` directive orders data in one or more tables using a special algorithm, similar to a Z-order function, that permits multicolumn I/O reduction.

  Consider using `BY INTERLEAVED ORDER` clustering when queries specify a variety of column combinations. For example, if queries of `sh.sales` specify different dimensions in different orders, then you can cluster data in the `sales` table according to columns in these dimensions.

Attribute clustering is only available for **[direct path INSERT](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-73D6FDFE-E38D-49F2-A5A5-B6BDB2FD297C)** operations. It is ignored for conventional DML.

### Advantages of Attribute-Clustered Tables

The primary benefit of attribute-clustered tables is I/O reduction, which can significantly reduce the I/O cost and CPU cost of table scans. I/O reduction occurs either with zones or by reducing physical I/O through closer physical proximity on disk for the clustered values.

An attribute-clustered table has the following advantages:

* You can cluster fact tables based on dimension columns in star schemas.

  In star schemas, most queries qualify dimension tables and not fact tables, so clustering by fact table columns is not effective. Oracle Database supports clustering on columns in dimension tables.


* I/O reduction can occur in several different scenarios:

 * When used with Oracle Exadata Storage Indexes, Oracle In-Memory min/max pruning, or zone maps
 
 * In OLTP applications for queries that qualify a prefix and use attribute clustering with linear order
 
 * On a subset of the clustering columns for `BY INTERLEAVED ORDER` clustering


* Attribute clustering can improve data compression, and in this way indirectly improve table scan costs.

  When the same values are close to each other on disk, the database can more easily compress them.


* Oracle Database does not incur the storage and maintenance cost of an index.

### Join Attribute Clustered Tables

Attribute clustering that is based on joined columns is called **join attribute clustering**. In contrast with table clusters, join attribute clustered tables do not store data from a group of tables in the same database blocks.

For example, consider an attribute-clustered table, `sales`, joined with a dimension table, `products`. The `sales` table contains only rows from the `sales` table, but the ordering of the rows is based on the values of columns joined from `products` table. The appropriate join is executed during data movement, direct path insert, and `CREATE TABLE AS SELECT` operations. In contrast, if `sales` and `products` were in a standard table cluster, the data blocks would contain rows from both tables.

### I/O Reduction Using Zones

A **zone** is a set of contiguous data blocks that stores the minimum and maximum values of relevant columns. When a SQL statement contains predicates on columns stored in a zone, the database compares the predicate values to the minimum and maximum stored in the zone to determine which zones to read during SQL execution.

I/O reduction is the ability to skip table or index blocks that do not contain data that the database needs to satisfy the query. This reduction can significantly reduce the I/O and CPU cost of table scans.

#### Zone Maps

A **zone map** is an independent access structure that divides data blocks into zones. Oracle Database implements each zone map as a type of **materialized view**.

Whenever `CLUSTERING` is specified on a table, the database automatically creates a zone map on the specified clustering columns. The zone map correlates minimum and maximum values of columns with consecutive data blocks in the attribute-clustered table. Attribute-clustered tables use zone maps to perform I/O reduction.

You can create attribute-clustered tables that do not use zone maps. You can also create zone maps without attribute-clustered tables. For example, you can create a zone map on a table whose rows are naturally ordered on a set of columns, such as a stock trade table whose trades are ordered by time. You execute DDL statements to create, drop, and maintain zone maps.

#### Zone Maps: Analogy

For a loose analogy of zone maps, consider a sales manager who uses a bookcase of pigeonholes, which are analogous to data blocks. Each pigeonhole has receipts (rows) describing shirts sold to a customer, ordered by ship date. In this analogy, a zone map is like a stack of index cards. Each card corresponds to a "zone" (contiguous range) of pigeonholes, such as pigeonholes 1-10. For each zone, the card lists the minimum and maximum ship dates for the receipts stored in the zone.

When someone wants to know which shirts shipped on a certain date, the manager flips the cards until she comes to the date range that contains the requested date, notes the pigeonhole zone, and then searches only pigeonholes in this zone for the requested receipts. In this way, the manager avoids searching every pigeonhole in the bookcase for the receipts.

#### Zone Maps: Example

This example illustrates how a zone map can prune data in a query whose predicate contains a constant.

Assume you create the following `lineitem` table:

```
CREATE TABLE lineitem 
  ( orderkey      NUMBER        , 
    shipdate      DATE          ,
    receiptdate   DATE          ,
    destination   VARCHAR2(50)  ,
    quantity      NUMBER        );
```

The table `lineitem` contains 4 data blocks with 2 rows per block. Table 2-2 shows the 8 rows of the table.

***Table 2-2 Data Blocks for lineitem Table***

| Block | orderkey | shipdate  | receiptdate | destination | quantity |
|:-----:|:--------:|:---------:|:-----------:|:------------|:--------:|
| 1     |  1       | 1-1-2014  | 1-10-2014   | San_Fran    | 100      |
| 1     |  2       | 1-2-2014  | 1-10-2014   | San_Fran    | 200      |
| 2     |  3       | 1-3-2014  | 1-9-2014    | San_Fran    | 100      |
| 2     |  4       | 1-5-2014  | 1-10-2014   | San_Diego   | 100      |
| 3     |  5       | 1-10-2014 | 1-15-2014   | San_Fran    | 100      |
| 3     |  6       | 1-12-2014 | 1-16-2014   | San_Fran    | 200      |
| 4     |  7       | 1-13-2014 | 1-20-2014   | San_Fran    | 100      |
| 4     |  8       | 1-15-2014 | 1-30-2014   | San_Jose    | 100      |

You can use the `CREATE MATERIALIZED ZONEMAP` statement to create a zone map on the `lineitem` table. Each zone contains 2 blocks and stores the minimum and maximum of the `orderkey`, `shipdate`, and `receiptdate` columns. Table 2-3 shows the zone map.

***Table 2-3 Zone Map for lineitem Table***

| Block Range | min orderkey | max orderkey  | min shipdate | max shipdate | min receiptdate | max receiptdate |
|:-----------:|:--------:|:---------:|:------------:|:-------------|:---------:|:--------:|
| 1-2         |  1       | 4         | 1-1-2014     | 1-5-2014     | 1-9-2014  | 1-10-2014 |
| 3-4         |  5       | 8         | 1-10-2014    | 1-15-2014    | 1-15-2014 | 1-30-2014 |

When you execute the following query, the database can read the zone map and then scan only blocks 1 and 2 because the date 1-3-2014 falls between the minimum and maximum dates:

`SELECT * FROM lineitem WHERE shipdate = '1-3-2014';`

### Attribute-Clustered Tables with Linear Ordering

A linear ordering scheme for a table divides rows into ranges based on user-specified attributes in a specific order. Oracle Database supports linear ordering on single or multiple tables that are connected through a primary-foreign key relationship.

For example, the `sales` table divides the `cust_id` and prod_id columns into ranges, and then clusters these ranges together on disk. When you specify the `BY LINEAR ORDER` directive for a table, significant I/O reduction can occur when a predicate specifies either the prefix column or all columns in the directive.

Assume that queries of `sales` often specify either a customer ID or a combination of a customer ID and product ID. You can create an attribute-clustered table so that such queries benefit from I/O reduction:

```
CREATE TABLE sales
(
   prod_id     NOT NULL NUMBER
,  cust_id     NOT NULL NUMBER
,  amount_sold NUMBER(10,2) ...
)
CLUSTERING 
  BY LINEAR ORDER (cust_id, prod_id)
  YES ON LOAD YES ON DATA MOVEMENT
  WITH MATERIALIZED ZONEMAP;
```

Queries that qualify both columns `cust_id` and `prod_id`, or the prefix `cust_id` experience I/O reduction. Queries that qualify `prod_id` only do not experience significant I/O reduction because `prod_id` is the suffix of the `BY LINEAR ORDER` clause. The following examples show how the database can reduce I/O during table scans.

***Example 2-8 Specifying Only cust_id***

An application issues the following query:

`SELECT * FROM sales WHERE cust_id = 100;`

Because the `sales` table is a `BY LINEAR ORDER` cluster, the database must only read the zones that include the `cust_id` value of `100`.

***Example 2-9 Specifying prod_id and cust_id***

An application issues the following query:

`SELECT * FROM sales WHERE cust_id = 100 AND prod_id = 2300;`

Because the `sales` table is a `BY LINEAR ORDER` cluster, the database must only read the zones that include the `cust_id` value of `100` and `prod_id` value of `2300`.

### Attribute-Clustered Tables with Interleaved Ordering

Interleaved ordering uses a technique that is similar to a Z-order. Interleaved ordering enables the database to prune I/O based on any subset of predicates in the clustering columns. Interleaved ordering is useful for dimensional hierarchies in a data warehouse.

As with attribute-clustered tables with linear ordering, Oracle Database supports interleaved ordering on single or multiple tables that are connected through a primary-foreign key relationship. Columns in tables other than the attribute-clustered table must be linked by foreign key and joined to the attribute-clustered table.

Large data warehouses frequently organize data in a **[star schema](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-2634049A-9DD8-4D5A-89BD-A6B72EF2C169)**. A **[dimension table](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-3089C278-6C49-4110-A10C-A95586E5868F)** uses a parent-child hierarchy and is connected to a **[fact table](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-B6DD258C-FFA5-4CDC-B5FB-98B3B9F20F99)** by a **[foreign key](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-EEE8A67D-6B0E-468E-9554-48FEE552BC9A)** (see "[Overview of Dimensions](https://docs.oracle.com/database/121/CNCPT/schemaob.htm#GUID-84D64C39-D8E8-4706-B4E3-519FE2CEE4C2)"). Clustering a fact table by interleaved order enables the database to use a special function to skip values in dimension columns during table scans.

***Example 2-10 Interleaved Ordering Example***

Suppose your data warehouse contains a `sales` fact table and its two dimension tables: `customers` and `products`. Most queries have predicates on the `customers` table hierarchy `(cust_state_province, cust_city)` and the products hierarchy `(prod_category, prod_subcategory)`. You can use interleaved ordering for the `sales` table as shown in the partial statement in the following example:

```
CREATE TABLE sales
(
   prod_id NUMBER NOT NULL
,  cust_id NUMBER NOT NULL
,  amount_sold NUMBER(10,2) ...
)
CLUSTERING sales
   JOIN products ON (sales.prod_id = products.prod_id)
   JOIN customers ON (sales.cust_id = customers.cust_id)
   BY INTERLEAVED ORDER
   (
     (  products.prod_category
     ,  products.prod_subcategory
     ),
     (  customers.cust_state_province
     ,  customers.cust_city
     )
   )
WITH MATERIALIZED ZONEMAP;
```

Suppose an application queries the `sales`, `products`, and `customers` tables in a join. The query specifies the `customers.prod_category` and `customers_cust_state_province` columns in the predicate as follows:

```
SELECT cust_city, prod_sub_category, SUM(amount_sold)
FROM   sales, products, customers
WHERE  sales.prod_id = products.prod_id 
AND    sales.cust_id = customers.cust_id
AND    customers.prod_category = 'Boys' 
AND    customers.cust_state_province = 'England - Norfolk' 
GROUP BY cust_city, prod_sub_category;
```

In the preceding query, the `prod_category` and `cust_state_province` columns are part of the clustering definition shown in the `CREATE TABLE` example. During the scan of the `sales` table, the database can consult the zone map and access only the rowids in this zone.

## Overview of Temporary Tables

A **temporary table** holds data that exists only for the duration of a transaction or session. Data in a temporary table is private to the session, which means that each session can only see and modify its own data.

### Temporary Table Creation

The `CREATE GLOBAL TEMPORARY TABLE` statement creates a temporary table. The `ON COMMIT` clause specifies whether the table data is transaction-specific (default) or session-specific. You create a temporary table for the database itself, not for every PL/SQL stored procedure.

You can create indexes for temporary tables with the `CREATE INDEX` statement. These indexes are also temporary. The data in the index has the same session or transaction scope as the data in the temporary table. You can also create a **[view](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-1274BCF5-2EC1-4752-B9CE-998A85A83307)** or **[trigger](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-8BA32C58-ACF3-4B6B-941F-586DE399D22A)** on a temporary table.

### Segment Allocation in Temporary Tables

Like permanent tables, temporary tables are persistent objects that are statically defined in the data dictionary. Temporary segments are allocated when a session first inserts data.

Until data is loaded in a session, the temporary table appears empty. For transaction-specific temporary tables, the database deallocates temporary segments at the end of the transaction. For session-specific temporary tables, the database deallocates temporary segments at the end of the session.

## Overview of External Tables

An **external table** accesses data in external sources as if this data were in a table in the database. You can use SQL, PL/SQL, and Java to query the external data.

External tables are useful for querying flat files. For example, a SQL-based application may need to access records in a text file. The records are in the following form:

```
100,Steven,King,SKING,515.123.4567,17-JUN-03,AD_PRES,31944,150,90
101,Neena,Kochhar,NKOCHHAR,515.123.4568,21-SEP-05,AD_VP,17000,100,90 
102,Lex,De Haan,LDEHAAN,515.123.4569,13-JAN-01,AD_VP,17000,100,90
```

You could create an external table, copy the file to the location specified in the external table definition, and use SQL to query the records in the text file.

External tables are also valuable for performing **[ETL](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-1145ABBB-D2C9-4321-B5E4-6435053062F0)** tasks common in **[data warehouse](https://docs.oracle.com/database/121/CNCPT/glossary.htm#GUID-0881E18F-D245-4980-8272-64964B64E3CC)** environments. For example, external tables enable the pipelining of the data loading phase with the transformation phase, eliminating the need to stage data inside the database in preparation for further processing inside the database. See "[Overview of Data Warehousing and Business Intelligence](https://docs.oracle.com/database/121/CNCPT/cmntopc.htm#GUID-E1451108-464E-4B5F-B173-11212278E308)".


### External Table Creation

Internally, creating an external table means creating metadata in the data dictionary. Unlike an ordinary table, an external table does not describe data stored in the database, nor does it describe how data is stored externally. Rather, external table metadata describes how the external table layer must *present* data to the database.

A `CREATE TABLE ... ORGANIZATION EXTERNAL` statement has two parts. The external table definition describes the column types. This definition is like a view that enables SQL to query external data without loading it into the database. The second part of the statement maps the external data to the columns.

External tables are read-only unless created with `CREATE TABLE AS SELECT` with the `ORACLE_DATAPUMP` access driver. Restrictions for external tables include no support for indexed columns, virtual columns, and column objects.

### External Table Access Drivers

An **access driver** is an API that interprets the external data for the database. The access driver runs inside the database, which uses the driver to read the data in the external table. The access driver and the external table layer are responsible for performing the transformations required on the data in the data file so that it matches the external table definition.

Figure 2-8 represents how external data is accessed.

***Figure 2-8 External Tables***
![Figure 2-8](https://docs.oracle.com/database/121/CNCPT/img/GUID-D1BE70CF-14E9-416C-9BF5-5FF7B0E6121E-default.gif)

Oracle provides the `ORACLE_LOADER` (default) and `ORACLE_DATAPUMP` access drivers for external tables. For both drivers, the external files are not Oracle data files.

`ORACLE_LOADER` enables read-only access to external files using SQL\*Loader. You cannot create, update, or append to an external file using the `ORACLE_LOADER` driver.

The `ORACLE_DATAPUMP` driver enables you to unload external data. This operation involves reading data from the database and inserting the data into an external table, represented by one or more external files. After external files are created, the database cannot update or append data to them. The driver also enables you to load external data, which involves reading an external table and loading its data into a database.

## Overview of Object Tables

An Oracle **object type** is a user-defined type with a name, attributes, and methods. An **object table** is a special kind of table in which each row represents an object. Object types make it possible to model real-world entities such as customers and purchase orders as objects in the database.

An object type defines a logical structure, but does not create storage. The following example creates an object type named `department_typ`:

```
CREATE TYPE department_typ AS OBJECT
   ( d_name     VARCHAR2(100),
     d_address  VARCHAR2(200) );
/
```

The following example creates an object table named `departments_obj_t` of the object type `department_typ`, and then inserts a row into the table. The attributes (columns) of the `departments_obj_t` table are derived from the definition of the object type.

```
CREATE TABLE departments_obj_t OF department_typ;
INSERT INTO departments_obj_t 
  VALUES ('hr', '10 Main St, Sometown, CA');
```

Like a relational column, an object table can contain rows of just one kind of thing, namely, object instances of the same declared type as the table. By default, every row object in an object table has an associated logical object identifier (OID) that uniquely identifies it in an object table. The OID column of an object table is a hidden column.

**[Documentation](https://docs.oracle.com/database/121/CNCPT/tablecls.htm#CNCPT88817)**

[![](https://docs.oracle.com/en/dcommon/img/oracle-doc-logo.png)](https://docs.oracle.com/en/)